In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import networkx as nx
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch import nn

import dgl
import dgl.nn.pytorch as dglnn
from dgl import apply_each


from qtaim_embed.utils.grapher import get_grapher
from qtaim_embed.data.molwrapper import mol_wrappers_from_df
from qtaim_embed.utils.tests import get_data
from qtaim_embed.core.dataset import HeteroGraphNodeLabelDataset
from qtaim_embed.models.layers import GraphConvDropoutBatch
from qtaim_embed.core.dataset import HeteroGraphNodeLabelDataset, Subset

In [2]:
# df = pd.read_pickle(
#    "/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_qtaim.pkl"
# )
# df.keys()

In [3]:
from qtaim_embed.utils.data import get_default_node_level_config

config = get_default_node_level_config()
config[
    "train_dataset_loc"
] = "/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_qtaim.pkl"
config["log_scale_features"] = True
config["log_scale_targets"] = True
config["debug"] = True

In [4]:
train_dataset = HeteroGraphNodeLabelDataset(
    # file="/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_full.pkl",
    file="/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_qtaim.pkl",
    allowed_ring_size=[3, 4, 5, 6, 7],
    allowed_charges=None,
    self_loop=True,
    extra_keys={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": [
            "extra_feat_bond_esp_total",
            "extra_feat_bond_ellip_e_dens",
            "extra_feat_bond_eta",
            "bond_length",
        ],
        "global": [],
    },
    target_dict={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": [
            "extra_feat_bond_esp_total",
            "extra_feat_bond_ellip_e_dens",
            "extra_feat_bond_eta",
        ],
    },
    extra_dataset_info={},
    debug=True,
    log_scale_features=True,
    log_scale_targets=True,
    standard_scale_features=True,
    standard_scale_targets=True,
)

... > running in debug mode
... > creating MoleculeWrapper objects


100%|██████████| 100/100 [00:00<00:00, 8435.00it/s]


... > bond_feats_error_count:  0
... > atom_feats_error_count:  0
element set {'N', 'H', 'C', 'O'}
selected atomic keys ['extra_feat_atom_esp_total']
selected bond keys ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta', 'bond_length']
selected global keys []
... > Building graphs and featurizing


100%|██████████| 100/100 [00:00<00:00, 316.92it/s]


included in labels
{'atom': ['extra_feat_atom_esp_total'], 'bond': ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta']}
included in graph features
{'atom': ['total_degree', 'total_H', 'is_in_ring', 'ring_size_3', 'ring_size_4', 'ring_size_5', 'ring_size_6', 'ring_size_7', 'chemical_symbol_N', 'chemical_symbol_H', 'chemical_symbol_C', 'chemical_symbol_O'], 'bond': ['metal bond', 'ring inclusion', 'ring size_3', 'ring size_4', 'ring size_5', 'ring size_6', 'ring size_7', 'bond_length'], 'global': ['num atoms', 'num bonds', 'molecule weight']}
original loader node types: dict_keys(['atom', 'bond', 'global'])
original loader label types: dict_keys([])
include names:  dict_keys(['atom', 'bond'])
... > parsing labels and features in graphs


100%|██████████| 100/100 [00:00<00:00, 13402.90it/s]


original loader node types: dict_keys(['atom', 'bond', 'global'])
original loader label types: dict_keys(['atom', 'bond'])
... > Log scaling features
... > Log scaling features complete
... > Scaling features
Standard deviation for feature 0 is 0.0, smaller than 0.001. You may want to exclude this feature.
... > Scaling features complete
... > feature mean(s): 
 {'atom': tensor([1.0335, 0.2958, 0.1931, 0.0230, 0.0399, 0.0681, 0.0430, 0.0191, 0.0456,
        0.3555, 0.2365, 0.0556]), 'bond': tensor([0.0000, 0.1958, 0.0242, 0.0442, 0.0701, 0.0459, 0.0208, 0.8152]), 'global': tensor([2.8185, 2.8375, 4.6976])}
... > feature std(s):  
 {'atom': tensor([0.3915, 0.4605, 0.3108, 0.1242, 0.1615, 0.2064, 0.1671, 0.1135, 0.1718,
        0.3465, 0.3286, 0.1882]), 'bond': tensor([0.0010, 0.3121, 0.1273, 0.1693, 0.2090, 0.1723, 0.1183, 0.0937]), 'global': tensor([0.1632, 0.1720, 0.0635])}
... > Log scaling targets
... > Log scaling targets complete
... > Scaling targets
... > Scaling targets complet

In [5]:
# TODO: build dataloader class

test_subset = Subset(train_dataset, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
test_subset.feature_size()

{'atom': 12, 'bond': 8, 'global': 3}

In [10]:
len_dict = {}
for key, value in test_subset.dataset.exclude_names.items():
    len_dict[key] = len(value)
len_dict

{'atom': 12, 'bond': 8, 'global': 3}

In [6]:
from qtaim_embed.data.dataloader import DataLoaderMoleculeNodeTask

In [7]:
dataloader = DataLoaderMoleculeNodeTask(train_dataset, batch_size=100, shuffle=True)

In [8]:
batch_graph, batch_label = next(iter(dataloader))

In [9]:
len_dict = train_dataset.feature_size()
atom_input_size = len_dict["atom"]
bond_input_size = len_dict["bond"]
global_input_size = len_dict["global"]
bond_output_size = 3
print(atom_input_size, bond_input_size, global_input_size)


n_heads = 1
hidden_feats = 64


class testmodel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = dglnn.HeteroGraphConv(
            {
                "a2b": GraphConvDropoutBatch(
                    in_feats=atom_input_size, out_feats=bond_input_size, dropout=0.2
                ),
                "b2a": GraphConvDropoutBatch(
                    in_feats=bond_input_size, out_feats=atom_input_size, dropout=0.2
                ),
                "a2g": GraphConvDropoutBatch(
                    in_feats=atom_input_size, out_feats=global_input_size, dropout=0.2
                ),
                "g2a": GraphConvDropoutBatch(
                    in_feats=global_input_size, out_feats=atom_input_size, dropout=0.2
                ),
                "b2g": GraphConvDropoutBatch(
                    in_feats=bond_input_size, out_feats=global_input_size, dropout=0.2
                ),
                "g2b": GraphConvDropoutBatch(
                    in_feats=global_input_size, out_feats=bond_input_size, dropout=0.2
                ),
                "a2a": GraphConvDropoutBatch(
                    in_feats=atom_input_size, out_feats=atom_input_size, dropout=0.2
                ),
                "b2b": GraphConvDropoutBatch(
                    in_feats=bond_input_size, out_feats=bond_input_size, dropout=0.2
                ),
                "g2g": GraphConvDropoutBatch(
                    in_feats=global_input_size, out_feats=global_input_size, dropout=0.2
                ),
            },
            aggregate="sum",
        )

        self.conv2 = dglnn.HeteroGraphConv(
            {
                "a2b": GraphConvDropoutBatch(
                    in_feats=atom_input_size,
                    out_feats=bond_output_size,
                ),
                "b2a": GraphConvDropoutBatch(
                    in_feats=bond_input_size,
                    out_feats=atom_input_size,
                ),
                "a2g": GraphConvDropoutBatch(
                    in_feats=atom_input_size,
                    out_feats=global_input_size,
                ),
                "g2a": GraphConvDropoutBatch(
                    in_feats=global_input_size,
                    out_feats=atom_input_size,
                ),
                "b2g": GraphConvDropoutBatch(
                    in_feats=bond_input_size,
                    out_feats=global_input_size,
                ),
                "g2b": GraphConvDropoutBatch(
                    in_feats=global_input_size,
                    out_feats=bond_output_size,
                ),
                "a2a": GraphConvDropoutBatch(
                    in_feats=atom_input_size,
                    out_feats=atom_input_size,
                ),
                "b2b": GraphConvDropoutBatch(
                    in_feats=bond_input_size,
                    out_feats=bond_output_size,
                ),
                "g2g": GraphConvDropoutBatch(
                    in_feats=global_input_size,
                    out_feats=global_input_size,
                ),
            },
            aggregate="sum",
        )
        """
        # get max feature length 
        self.max_feature_len = max([atom_input_size, bond_input_size, global_input_size])
        
        self.conv3 = dglnn.HeteroGraphConv(
            {
                "b2a": dglnn.GATConv(
                    in_feats=self.max_feature_len, 
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "g2a": dglnn.GATConv(
                    in_feats=self.max_feature_len, 
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "a2a": dglnn.GATConv(
                    in_feats=self.max_feature_len, 
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "g2b": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "a2b": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats ,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "b2b": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),

                "b2g": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "g2g": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "a2g": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats= hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
            },
            aggregate="sum",
        )

        self.conv4 = dglnn.HeteroGraphConv(
            {
                "b2a": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads, 
                    out_feats=atom_input_size,
                    #aggregator_type="lstm",
                ),
                "g2a": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads, 
                    out_feats=atom_input_size,
                    #aggregator_type="lstm",
                ),
                "a2a": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads, 
                    out_feats=atom_input_size,
                    #aggregator_type="lstm",
                ),
                "g2b": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=bond_output_size,
                    #aggregator_type="lstm",
                ),
                "a2b": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=bond_output_size,
                    #aggregator_type="lstm",
                ),
                "b2b": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=bond_output_size,
                    #aggregator_type="lstm",
                ),

                "b2g": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=global_input_size,
                    #aggregator_type="lstm",
                ),
                "g2g": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=global_input_size,
                    #aggregator_type="lstm",
                ),
                "a2g": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats= global_input_size,
                ),
            },
            aggregate="sum",
        )
        """

    def forward(self, graph, inputs):
        """
        # feats = self.conv(graph, inputs)
        # feats = self.conv2(graph, feats)
        #print(inputs.keys())
        #for i in inputs.keys():
        #    print(i)
        #    print(inputs[i].shape)
        # zero pad with typed linear to get max feature length for each feature
        inputs = {k: F.pad(v, (0, self.max_feature_len - v.shape[1])) for k, v in inputs.items()}
        feats = self.conv3(graph, inputs)
        #print("pass" * 30)
        feats = {k: torch.reshape(F.relu(v), (v.shape[0], -1)) for k, v in feats.items()}
        feats = self.conv4(graph, feats)
        return feats
        """

        # print(inputs)
        feats = self.conv(graph, inputs)
        # print(feats)
        feats = self.conv2(graph, feats)
        return feats


model = testmodel()

12 8 3


In [10]:
graph = batch_graph
forward_out = model(graph, graph.ndata["feat"])

In [11]:
from qtaim_embed.models.node_level.base_gnn import GCNNodePred

model_imported = GCNNodePred(
    atom_input_size=atom_input_size,
    bond_input_size=bond_input_size,
    global_input_size=global_input_size,
    n_conv_layers=2,
    target_dict={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": [
            "extra_feat_bond_esp_total",
            "extra_feat_bond_ellip_e_dens",
            "extra_feat_bond_eta",
        ],
    },
    dropout=0.2,
)

number of output dims 4


In [14]:
dataloader = DataLoaderMoleculeNodeTask(train_dataset, batch_size=100, shuffle=True)
batch_graph, batch_label = next(iter(dataloader))
model_imported.forward(batch_graph, batch_label)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1597x1 and 12x12)

In [16]:
from torch.nn import functional as F
from sklearn.metrics import r2_score

# from tqdm import tqdm
import tqdm.notebook as tq

opt = torch.optim.Adam(model.parameters(), lr=0.01)


for epoch in range(50):
    training_loss_list = []
    with tqdm(dataloader) as tq:
        model_imported.train()
        r2_list = []
        tq.set_description(f"Epoch {epoch+1}")
        training_loss = 0
        target_type = "bond"
        for step, (batch_graph, batch_label) in enumerate(tq):
            # forward propagation by using all nodes and extracting the user embeddings
            batch_graph, batch_label = next(iter(dataloader))
            labels = batch_label[target_type]
            logits = model_imported(batch_graph, batch_graph.ndata["feat"])[target_type]
            # print(logits.shape)
            # print(labels.shape)
            # compute loss
            loss = F.mse_loss(logits, labels)
            training_loss_list.append(loss.item())
            # loss_mae = F.l1_loss(logits, labels)
            # compute r2 score
            r2 = r2_score(logits.detach().numpy(), labels.detach().numpy())
            r2_list.append(r2)
            # Compute validation accuracy.  Omitted in this example.
            # backward propagation
            opt.zero_grad()
            loss.backward()
            opt.step()
            training_loss += loss.item()
            # tq.set_postfix({"Step": step, "MSE": loss.item()})

        r2_mean = np.mean(r2_list)
        loss = np.mean(training_loss_list)
        tq.set_postfix({"final_t_loss": training_loss, "R_2": r2_mean})
        print(r2_mean, loss)

        # tq.update()
        tq.close()

Epoch 1: 100%|██████████| 1/1 [00:00<00:00, 45.06it/s]


-18.491557544946957 64.81822204589844


Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 53.65it/s]


-19.652633210662124 64.9208755493164


Epoch 3: 100%|██████████| 1/1 [00:00<00:00, 51.62it/s]


...

-18.350620250137013 64.65076446533203


Epoch 49: 100%|██████████| 1/1 [00:00<00:00, 54.05it/s]


-19.139153756222623 65.30514526367188


Epoch 50: 100%|██████████| 1/1 [00:00<00:00, 51.83it/s]


-18.529322300103342 65.0848617553711


In [43]:
from torch.nn import functional as F
from sklearn.metrics import r2_score

# from tqdm import tqdm
import tqdm.notebook as tq

opt = torch.optim.Adam(model.parameters(), lr=0.01)


for epoch in range(50):
    training_loss_list = []
    with tqdm(dataloader) as tq:
        model.train()
        r2_list = []
        tq.set_description(f"Epoch {epoch+1}")
        training_loss = 0
        target_type = "bond"
        for step, (batch_graph, batch_label) in enumerate(tq):
            # forward propagation by using all nodes and extracting the user embeddings
            batch_graph, batch_label = next(iter(dataloader))
            labels = batch_label[target_type]
            logits = model(batch_graph, batch_graph.ndata["feat"])[target_type]
            # print(logits.shape)
            # print(labels.shape)
            # compute loss
            loss = F.mse_loss(logits, labels)
            training_loss_list.append(loss.item())
            # loss_mae = F.l1_loss(logits, labels)
            # compute r2 score
            r2 = r2_score(logits.detach().numpy(), labels.detach().numpy())
            r2_list.append(r2)
            # Compute validation accuracy.  Omitted in this example.
            # backward propagation
            opt.zero_grad()
            loss.backward()
            opt.step()
            training_loss += loss.item()
            # tq.set_postfix({"Step": step, "MSE": loss.item()})

        r2_mean = np.mean(r2_list)
        loss = np.mean(training_loss_list)
        tq.set_postfix({"final_t_loss": training_loss, "R_2": r2_mean})
        print(r2_mean, loss)

        # tq.update()
        tq.close()

Epoch 1: 100%|██████████| 1/1 [00:00<00:00, 23.45it/s]


-3.7205302931984163 14.68019962310791


Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 31.44it/s]


-3.9273690115274005 14.689446449279785


Epoch 3: 100%|██████████| 1/1 [00:00<00:00, 21.65it/s]


...

-2.2101164541132303 11.880921363830566


Epoch 49: 100%|██████████| 1/1 [00:00<00:00, 26.01it/s]


-2.021475935807809 11.253496170043945


Epoch 50: 100%|██████████| 1/1 [00:00<00:00, 24.18it/s]

-2.181490068655195 12.423982620239258


In [32]:
label_list = []
predictions_list = []

with tqdm(dataloader) as tq, torch.no_grad():
    for step, (batch_graph, batch_label) in enumerate(tq):
        batch_graph, batch_label = next(iter(dataloader))
        labels = batch_label[target_type]
        logits = model(batch_graph, batch_graph.ndata["feat"])[target_type]
        label_list.append(labels.cpu().numpy())
        predictions_list.append(logits.cpu().numpy())


cat_labels = np.concatenate(label_list)
cat_preds = np.concatenate(predictions_list)

100%|██████████| 1/1 [00:00<00:00, 28.51it/s]


In [16]:
print(cat_labels.shape)
print(cat_preds.shape)

(1632, 1)
(1632, 1)


In [33]:
r2 = r2_score(cat_labels[:, 0], cat_preds[:, 0])
print(r2)

0.8445300518290597


In [34]:
r2 = r2_score(cat_labels[:, 1], cat_preds[:, 1])
print(r2)

0.6478962999038074


In [35]:
r2 = r2_score(cat_labels[:, 2], cat_preds[:, 2])
print(r2)

0.43115008510261954


In [5]:
from qtaim_embed.core.datamodule import QTAIMNodeTaskDataModule
from qtaim_embed.models.node_level.base_gcn import GCNNodePred

dm = QTAIMNodeTaskDataModule(config=config)

In [6]:
import pytorch_lightning as pl

torch.set_float32_matmul_precision("high")
len_dict = dm.setup(stage="fit")
atom_input_size = len_dict["atom"]
bond_input_size = len_dict["bond"]
global_input_size = len_dict["global"]

... > running in debug mode
... > creating MoleculeWrapper objects


100%|██████████| 100/100 [00:00<00:00, 7746.14it/s]


... > bond_feats_error_count:  0
... > atom_feats_error_count:  0
element set {'N', 'H', 'C', 'O'}
selected atomic keys ['extra_feat_atom_esp_total']
selected bond keys ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta', 'bond_length']
selected global keys []
... > Building graphs and featurizing


100%|██████████| 100/100 [00:00<00:00, 335.27it/s]


included in labels
{'atom': ['extra_feat_atom_esp_total'], 'bond': ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta']}
included in graph features
{'atom': ['total_degree', 'total_H', 'is_in_ring', 'ring_size_3', 'ring_size_4', 'ring_size_5', 'ring_size_6', 'ring_size_7', 'chemical_symbol_N', 'chemical_symbol_H', 'chemical_symbol_C', 'chemical_symbol_O'], 'bond': ['metal bond', 'ring inclusion', 'ring size_3', 'ring size_4', 'ring size_5', 'ring size_6', 'ring size_7', 'bond_length'], 'global': ['num atoms', 'num bonds', 'molecule weight']}
original loader node types: dict_keys(['atom', 'bond', 'global'])
original loader label types: dict_keys([])
include names:  dict_keys(['atom', 'bond'])
... > parsing labels and features in graphs


100%|██████████| 100/100 [00:00<00:00, 14270.22it/s]


original loader node types: dict_keys(['atom', 'bond', 'global'])
original loader label types: dict_keys(['atom', 'bond'])
... > Log scaling features
... > Log scaling features complete
... > Scaling features
Standard deviation for feature 0 is 0.0, smaller than 0.001. You may want to exclude this feature.
... > Scaling features complete
... > feature mean(s): 
 {'atom': tensor([1.0335, 0.2958, 0.1931, 0.0230, 0.0399, 0.0681, 0.0430, 0.0191, 0.0456,
        0.3555, 0.2365, 0.0556]), 'bond': tensor([0.0000, 0.1958, 0.0242, 0.0442, 0.0701, 0.0459, 0.0208, 0.8152]), 'global': tensor([2.8185, 2.8375, 4.6976])}
... > feature std(s):  
 {'atom': tensor([0.3915, 0.4605, 0.3108, 0.1242, 0.1615, 0.2064, 0.1671, 0.1135, 0.1718,
        0.3465, 0.3286, 0.1882]), 'bond': tensor([0.0010, 0.3121, 0.1273, 0.1693, 0.2090, 0.1723, 0.1183, 0.0937]), 'global': tensor([0.1632, 0.1720, 0.0635])}
... > Log scaling targets
... > Log scaling targets complete
... > Scaling targets
... > Scaling targets complet

In [7]:
print(atom_input_size, bond_input_size, global_input_size)

12 8 3


In [8]:
model = GCNNodePred(
    atom_input_size=atom_input_size,
    bond_input_size=bond_input_size,
    global_input_size=global_input_size,
    target_dict={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": [
            "extra_feat_bond_esp_total",
            "extra_feat_bond_ellip_e_dens",
            "extra_feat_bond_eta",
        ],
    },
    activation="ReLU",
    conv_fn="ResidualBlock",
    resid_n_graph_convs=3,
    dropout=0.2,
    bias=True,
    batch_norm=True,
    n_conv_layers=8,
    lr_plateau_patience=10,
    lr=0.02,
    weight_decay=0.0001,
)

triggered normal outer layer
triggered normal outer layer
triggered normal outer layer
triggered normal outer layer
triggered normal outer layer
triggered normal outer layer
triggered output_layer args
triggered early stop condition!!!
target_dict {'atom': ['extra_feat_atom_esp_total'], 'bond': ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta']}
triggered separate intermediate layer
triggered separate intermediate layer
triggered separate outer layer
number of output dims 4


In [10]:
trainer_transfer = pl.Trainer(
    max_epochs=100,
    accelerator="gpu",
    devices=1,
    enable_progress_bar=True,
    gradient_clip_val=3.0,
    default_root_dir="./test/",
    precision="32",
    log_every_n_steps=10,
)

trainer_transfer.fit(model, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name            | Type               | Params
--------------------------------------------------------
0  | conv_layers     | ModuleList         | 6.1 K 
1  | loss            | MultioutputWrapper | 0     
2  | train_r2        | MultioutputWrapper | 0     
3  | train_torch_l1  | MultioutputWrapper | 0     
4  | train_torch_mse | MultioutputWrapper | 0     
5  | val_r2          | MultioutputWrapper | 0     
6  | val_torch_l1    | MultioutputWrapper | 0     
7  | val_torch_mse   | MultioutputWrapper | 0     
8  | test_r2         | MultioutputWrapper | 0     
9  | test_torch_l1   | MultioutputWrapper | 0     
10 | test_torch_mse  | MultioutputWrapper | 0     
--------------------------------------------------------
6.1 K     Trainable params
0         Non-trainable params
6.1 K     Total par

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 00100: reducing learning rate of group 0 to 3.9063e-05.


In [14]:
dm.train_dataloader().dataset[0].ndata["labels"]["atom"].shape

torch.Size([22, 1])

In [15]:
# get example output from model
model.eval()
out = model(
    dm.train_dataloader().dataset[0], dm.train_dataloader().dataset[0].ndata["feat"]
)

RuntimeError: The size of tensor a (3) must match the size of tensor b (8) at non-singleton dimension 1

In [13]:
out["bond"].shape

torch.Size([22, 8])

In [12]:
out["atom"].shape

torch.Size([22, 12])

In [54]:
dm.train_dataloader().dataset[0].ndata["feat"]["atom"].shape

torch.Size([22, 12])